# Agent-based model Quickstart guide

# Basic ABM model

## Imports

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from abm.model import *
from abm.plotting import *
from abm.parallel import *
from abm.population import *
from abm.characteristics import load_population_dataset
from optimization.operators import *

## Running the simulation

### Running a basic simulation

We must first set up the model's parameters, using a dictionary:

In [2]:
params = {
        'inf_params': {'age': 0.000},
        'test_params': {'age': 0.000},
        'inf_fraction_param': 30,
        'inf_lvl_error_term': -15,
        'inf_proba_sigmoid_slope': 1.0,
        'test_inf_lvl_param': 0.1,
        'test_error_term': -15,
        'test_proba_sigmoid_slope': 10.0,
        'recovery_mean_time': 8.0,
        'recovery_std_time': 2.0,
        'Restriction_begin': 0,
        'Restriction_end': 9
}

We then need to set up the initial conditions. To do so, we need to define a fixed number of infections for the first few days of the simulation. The infections created during this phase will be spread uniformly within the population:

In [3]:
# Defines a linearly increasing initial infections curve, for the first 10 days.
initial_infections = np.arange(2) * 10 +10
print(initial_infections)

[10 20]


We can now create the ```ABM``` object, which will load the mobility data. The default data paths need to be specified in ```simulation_config.yml```.

In [4]:
abm = ABM(params)
# Initializes the simulation
abm.force_simulation_start(initial_infections)

Loading activity data...
Processing period 0
Processing period 1
Processing period 2
Processing period 3
Processing period 4
Processing period 5
Processing period 6
Processing period 7
Processing period 8
Processing period 9
Processing period 10
Processing period 11
Processing period 12
Processing period 13
Processing period 14
Processing period 15
Processing period 16
Processing period 17
Processing period 18
Processing period 19
Processing period 20
Processing period 21
Processing period 22
Processing period 23
Loading reduction schedules ...
Loading activity data...
Processing period 0
Processing period 1
Processing period 2
Processing period 3
Processing period 4
Processing period 5
Processing period 6
Processing period 7
Processing period 8
Processing period 9
Processing period 10
Processing period 11
Processing period 12
Processing period 13
Processing period 14
Processing period 15
Processing period 16
Processing period 17
Processing period 18
Processing period 19
Processing per

We can finally run the simulation:

In [5]:
simulation_days = 9
abm.run_simulation(simulation_days, verbose=True)

Day  0


/Users/antoinedebouille/Documents/EPFL/MASTER/MA2/Semester project/ABM model/Covid_restriction_Github_version/abm/model.py:249: RuntimeWarning: invalid value encountered in divide
  infected_fractions = infected_visitors / visitors


Day  1
Day  2
Day  3
Day  4
Day  5
Day  6
Day  7
Day  8
Day  9
Simulation ended. 


The results can be retrieved using ```abm.results.get_daily_results()``` (for daily-aggregated results) or ```abm.results.get_per_period_results()``` (for the simulation state after every single period):

In [6]:
results = abm.results.get_daily_results()
results.head(115)

,daily summed new infections,daily summed infected agents,daily summed recovered agents,daily summed tests,daily summed positive tests,daily avg new infections,daily avg infected agents,daily avg recovered agents,daily avg tests,daily avg positive tests,day
day,,,,,,,,,,,
0,18,207,0,0,0,0.750000,8.625000,0.000000,0.0,0.0,0
1,7,526,0,0,0,0.291667,21.916667,0.000000,0.0,0.0,1
2,38,1329,0,0,0,1.583333,55.375000,0.000000,0.0,0.0,2
3,12,1648,0,0,0,0.500000,68.666667,0.000000,0.0,0.0,3
4,23,2051,17,0,0,0.958333,85.458333,0.708333,0.0,0.0,4
5,2,2313,57,0,0,0.083333,96.375000,2.375000,0.0,0.0,5
6,17,2489,118,0,0,0.708333,103.708333,4.916667,0.0,0.0,6
7,12,2662,259,0,0,0.500000,110.916667,10.791667,0.0,0.0,7
8,9,2778,463,0,0,0.375000,115.750000,19.291667,0.0,0.0,8


In [7]:
abm.results.get_per_period_results().head()

,new infections,infected agents,recovered agents,tests,positive tests,period,day
0,1,1,0,0,0,0,0
1,0,1,0,0,0,1,0
2,2,3,0,0,0,2,0
3,0,3,0,0,0,3,0
4,0,3,0,0,0,4,0


Finally, the ```ABM_Plotter``` class has predefined methods to display the results:

In [8]:
plotter = ABM_Plotter(abm)
# Displays the new infections and positive tests, and saves the image.
plotter.plot_curves(show_fig=True, save_img_to="tests/figures/simulation.jpg")

In [ ]:
# For the following display, we need to load the population dataset, which contains the agents'
# homes' geographical coordinates:
population_dataset = load_population_dataset()
plotter.plot_infection_spread(population_dataset, show_fig=True)

# Adding mobility reductions in the simulation and running VNS

### To do list: 
This code selects for each segmentation a schedule in the set of schedules possible.

The optimization process is therefore:

$Min  L (T_{begin}, T_{end}, M)$ 
Where M is a mapping between each age group and each scenario : each group will have a scenario for each iteration of the VND


- Adapter pour renvoyer le nombre de morts par jour
- Vérifier que la solution est valide
- Créer des scénarios


#### Scénarios
1) Faire des scénarios en faisant tourner la fonction '20230323_prepare_data.ipynb'

#### Preprocessing : OK
1) A chaque itération, préprocessing des schedules possibles pour chaque segmentation : ok
2) Envoi dans du dictionnaire M pour merge les schedules : ok
3) Envoi du schecule de base : ok


#### ABM
1) Adapter pour renvoyer le nombre de morts par jour
2) Adapter pour renvoyer le nombre d'heure confiné par segmentation pour pouvoir calculer un poids. (nombre de jour confiné * (calendrier entier - calendrier de la segmentation)) ok 
3) Adapter ABM pour avoir 2 calendriers : un pour le confinement, un pour le normal: fonction load_activity_policy : ok
4) Itérer sur les jours : si la période appartient à la restriction, on applique le calendrier de la restriction, sinon on applique le calendrier normal : ok
5) Changer la class population pour ajouter un attribut 'restriction' qui sera le calendrier durant la restriction : ok


#### Optimisation
1) Créer les opérateurs : temporels, changer de schedule pour chaque segmentation : ok
2) Vérifier que la solution est valide

## Preprocessing

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from abm.model import *
from abm.plotting import *
from abm.parallel import *
from abm.population import *
from abm.characteristics import load_population_dataset
from optimization.operators import *


In [6]:
def prepare_calendars(sol2):
    """Prepares the calendars for the simulation and stores it in "data/abm/vaud/prepared/temp_period_activities"
    sol2 : list of scenarios numbers for each segment

    :return: Number of periods for each activity
    """
    #Merge csv files for each segmantation according to the solution
    segments = ['0 - 9', '10 - 19', '20 - 29', '30 - 39', '40 - 49', '50 - 59', '60 - 69', '70 - 79', '80+']
    for i in range(len(sol2)):
        folder = "data/abm/vaud/prepared/scenarios/scenario_" + str(sol2[i]) + "/"
        file = folder + "vaud_period_activities_" + segments[i] + ".csv.gz"
        temp = pd.read_csv(file)
        if i == 0:
            df = temp
        else: 
            df = pd.concat([df, temp])
    
    unique_periods = df['period'].unique()
    activities = df['type'].unique()
    # Count the number of periods for each activity
    count_activities = dict()
    for activity in activities:
        count_activities[activity] = len(df[df['type'] == activity])

    #Applies preprocessing and saves the restriction schecule
    _AGENTS_ID_TRANSLATIONS_FILE_ = "data/abm/vaud/prepared/vaud_agents_id_translations.csv.gz"
    _FACILITIES_ID_TRANSLATIONS_FILE_ = "data/abm/vaud/prepared/vaud_facilities_id_translations.csv.gz"
    _PERIOD_ACTIVITIES_REP_ = "data/abm/vaud/prepared/temp_period_activities/"
    
    agents_translations = pd.read_csv(_AGENTS_ID_TRANSLATIONS_FILE_, index_col=0)
    agents_translations['agent_index']=agents_translations['agent_index'].astype(int)
    facilities_translations = pd.read_csv(_FACILITIES_ID_TRANSLATIONS_FILE_, index_col=0)
    facilities_translations['facility_index']=facilities_translations['facility_index'].astype(int)
    

    for period_index, period in enumerate(unique_periods):
        print("Processing period ", period)
        # Isolate the activities that occurred during that period
        sub_activ = df[df['period'] == period]
        # Translate the agent ids to agent index
        sub_activ = sub_activ.merge(agents_translations, left_on="id", right_index=True)
        # Translate the facility names to indexes
        sub_activ = sub_activ.merge(facilities_translations, left_on="facility", right_index=True)
        # Only keep the relevant info
        sub_activ = sub_activ.drop(['age', 'period', 'id', 'facility'], axis=1)
        # Save the sub dataset
        sub_activ.to_csv(os.path.join(_PERIOD_ACTIVITIES_REP_, f"{str(period_index)}.csv.gz"),
                        index=False)
        
    return count_activities

### Define the problem

Change to adapt to ABM model

In [7]:
class optimisationcovid(vns.problemClass):
    """Class defining the covid policy problem. Note the inheritance from the
    abstract class used by the VNS algorithm. It guarantees the
    compliance with the requirements of the algorithm.

    """

    def __init__(self, costs,normal_gdp,N,lengthsimulation,n_scenarios, initial_infections, average_remaining_working_years, proportion_work_infection,cost_healthcare, severeforms,proportion_remoteless,reduction_efficiency_remote_work, params):
        """Ctor"""
        super().__init__()
        self.cost_activities=costs
        self.average_remaining_working_years = average_remaining_working_years # Number of years to work
        self.proportion_work_infection = proportion_work_infection
        self.proportion_remoteless = proportion_remoteless
        self.reduction_efficiency_remote_work = reduction_efficiency_remote_work
        self.cost_healthcare = cost_healthcare # Cost of healthcare per day
        self.severeforms = severeforms #Pourcentage of severe forms in the positive tests
        self.params = params
        self.normal_gdp = normal_gdp
        self.N= N
        self.n_scenarios = n_scenarios
        self.lengthsimulation=lengthsimulation
        self.initial_infections = initial_infections,
        self.operators = {
            'Increase_policy_end': increase_policy_end,
            'Decrease_policy_end': decrease_policy_end,
            'Increase_policy_beginning': increase_policy_beginning,
            'Decrease_policy_beginning': decrease_policy_beginning,
            'Shift_later': shift_later,
            'Shift_earlier':shift_earlier,
            'decrease_scenario_0_9': decrease_scenario_0_9,
            'increase_scenario_0_9': increase_scenario_0_9,
            'decrease_scenario_10_19': decrease_scenario_10_19,
            'increase_scenario_10_19': increase_scenario_10_19,
            'decrease_scenario_20_29': decrease_scenario_20_29,
            'increase_scenario_20_29': increase_scenario_20_29,
            'decrease_scenario_30_39': decrease_scenario_30_39,
            'increase_scenario_30_39': increase_scenario_30_39,
            'decrease_scenario_40_49': decrease_scenario_40_49,
            'increase_scenario_40_49': increase_scenario_40_49,
            'decrease_scenario_50_59': decrease_scenario_50_59,
            'increase_scenario_50_59': increase_scenario_50_59,
            'decrease_scenario_60_69': decrease_scenario_60_69,
            'increase_scenario_60_69': increase_scenario_60_69,
            'decrease_scenario_70_79': decrease_scenario_70_79,
            'increase_scenario_70_79': increase_scenario_70_79,
            'decrease_scenario_80_plus': decrease_scenario_80_plus,
            'increase_scenario_80_plus': increase_scenario_80_plus        
        }
        self.operatorsManagement = vns.operatorsManagement(
            self.operators.keys()
        )
        self.currentSolution = None
        self.lastOperator = None
        

    def startsolution(self):
        """
        :return: the first sceneario for each segment (9 age segments), and counts the number of activities in the no-restriction schedules
        :rtype: class onesolution
        """        
        _PERIOD_ACTIVITIES_ = 'data/abm/vaud/prepared/vaud_period_activities.csv.gz'    
        df = pd.read_csv(_PERIOD_ACTIVITIES_, index_col=0)
        activities = df['type'].unique()    
        self.count_activities = dict()
        for activity in activities:
            self.count_activities[activity] = len(df[df['type'] == activity])
        z=np.array([0,self.lengthsimulation,1,1,1,1,1,1,1,1,1])
        return onesolution(z)

    def isValid(self, aSolution):
        """Check if the policy is feasible

        :param aSolution: solution to check
        :type aSolution: class onesolution

        :return: True if the policy is valid
        :rtype: bool
        """
        sol = aSolution.x
        for i in range(2,11):
            if sol[i]>self.n_scenarios:
                return False, 'Scenario must be less than the number of scenarios'
            if sol[i]<1:
                return False, 'Scenario must be greater than 0'
        if sol[1] > self.lengthsimulation:
            return False, 'Policy ends after the simulation ends'
        if sol[0] < 0 :
            return False, 'Policy starts before the simulation starts'
        if sol[1]-sol[0]<3:
            return False, 'Policy must last at least 3 days'
        return True, 'Feasible policy'

    def evaluate(self, aSolution):

        """ Creates a ABM model and evalautes

        :param aSolution: solution to evaluate
        :type aSolution: class onesolution

        """
        sol=aSolution.x
        #if self.isValid(aSolution)[0]:
        solc = copy.deepcopy(sol)
        sol2 = list(solc[2:])

        #Call preprocessing function to obtain the restriction schedule according to the solution and count the number of period per activity
        self.restriction_count_activities = prepare_calendars(sol2)
        #Initializes the simulation and forces initial infections
        self.abm = ABM(self.params)
        self.abm.force_simulation_start(self.initial_infections)
        # Runs the simulation with restriction period
        self.abm.set_param('Restriction_begin', solc[0])
        self.abm.set_param('Restriction_end', solc[1] )
        self.abm.force_simulation_start(self.initial_infections)
        self.abm.run_simulation(self.lengthsimulation, verbose=True)
        
        # Compute I and D
        results = self.abm.results.get_daily_results()
        I = results["daily summed new infections"] # daily new infections
        D = results["daily summed new infections"] # We should replace by number of death 
        totalI= I.sum()
        totalI.round(0)
        totalD= D.sum()
        totalD.round(2)

        # Duration of the policy
        Deltat = solc[1]-solc[0] 
        # Number of confined activities
        count_confined_activities = dict()
        C_policies = 0
        for activity in self.count_activities.keys():
            if activity in self.restriction_count_activities.keys():
                count_confined_activities[activity] = Deltat*(self.count_activities[activity] - self.restriction_count_activities[activity])
                C_policies += count_confined_activities[activity]*self.cost_activities[activity]
            else:
                pass
        C_policies = self.proportion_remoteless * self.reduction_efficiency_remote_work * C_policies

        #Loss
        sanitary_cost = (self.severeforms * totalD + totalI).round(2)
        C_death = self.severeforms * totalD * self.normal_gdp / self.N *self.average_remaining_working_years 
        C_infection = self.proportion_work_infection * self.normal_gdp / self.N / 365 * totalI * 5 # 5 is the average number of days of work lost per infection - should adapt with the model
        C_helthcare = self.cost_healthcare * self.severeforms * totalI 
        economic_cost = (C_death + C_infection + C_helthcare + C_policies).round(2)/1000
        
        aSolution.objectives = [sanitary_cost, economic_cost]
        
        print("EVALUATE")
        print(aSolution.objectives)
    #else:
        #    aSolution.objectives = [np.inf, np.inf]


    def describe(self, aSolution):
        """Short description of a solution

        :param aSolution: solution to describe
        :type aSolution: class onesolution

        :return: description
        :rtype: str
        """
        sol = aSolution.x
        print(f"{sol[1]}")
        for x in sol[2:]:
            print("Scenario n° ",x)
        return str(aSolution)

    def generateNeighbor(self, aSolution, neighborhoodSize):
        """Generate a neighbor from the negihborhood of size
        ``neighborhoodSize``using one of the operators

        :param aSolution: current solution
        :type aSolution: class onesolution

        :param neighborhoodSize: size of the neighborhood
        :type neighborhoodSize: int

        :return: number of modifications actually made
        :rtype: int

        """
        #print('on est dans generateNeighbor')
        # Select one operator.
        self.lastOperator = self.operatorsManagement.selectOperator()
        return self.applyOperator(
            aSolution, self.lastOperator, neighborhoodSize
        )

    def neighborRejected(self, aSolution, aNeighbor):
        """Informs the operator management object that the neighbor has been
        rejected.

        :param aSolution: current solution
        :type aSolution: class onesolution

        :param aNeighbor: proposed neighbor
        :type aNeighbor: class onesolution

        :raise biogemeError: if no operator has been used yet.
        """
        if self.lastOperator is None:
            raise excep.biogemeError('No operator has been used yet.')
        self.operatorsManagement.decreaseScore(self.lastOperator)

    def neighborAccepted(self, aSolution, aNeighbor):
        """Informs the operator management object that the neighbor has been
        accepted.

        :param aSolution: current solution
        :type aSolution: class onesolution

        :param aNeighbor: proposed neighbor
        :type aNeighbor: class onesolution

        :raise biogemeError: if no operator has been used yet.
        """
        if self.lastOperator is None:
            raise excep.biogemeError('No operator has been used yet.')
        self.operatorsManagement.increaseScore(self.lastOperator)

    def applyOperator(self, solution, name, size=1):
        """Apply a specific operator on a solution, using a neighborhood of
        size ``size``

        :param solution: current solution
        :type solution: class onesolution

        :param name: name of the operator
        :type name: str

        :param size: size of the neighborhood
        :type size: int

        :return: number of modifications actually made
        :rtype: int

        :raise biogemeError: if the name of the operator is unknown.

        """
        op = self.operators.get(name)
        if op is None:
            raise excep.biogemeError(f'Unknown operator: {name}')
        return op(solution, size) 

### Running the VNS

In [8]:
params = {
        'inf_params': {'age': 0.000},
        'test_params': {'age': 0.000},
        'inf_fraction_param': 30,
        'inf_lvl_error_term': -15,
        'inf_proba_sigmoid_slope': 1.0,
        'test_inf_lvl_param': 0.1,
        'test_error_term': -15,
        'test_proba_sigmoid_slope': 10.0,
        'recovery_mean_time': 8.0,
        'recovery_std_time': 2.0,
        'Restriction_begin': 1,
        'Restriction_end': 6}

#### Faire tourner pour l'age (3-4 catégories) et un autre (voitures, unemployed, ...)

In [9]:
costs = dict({'home': 0, 'shop': 2, 'leisure': 1.5, 'work': 17.2, 'other': 2, 'education': 17.2})
normal_gdp = 62000000
N = 814000
lengthsimulation = 20
initial_infections = (np.arange(5)+1) * 100
n_scenarios = 4
average_remaining_working_years = 20 # Number of years to work
proportion_work_infection = 0.4 # Proportion of the cost of infection that is due to the loss of productivity
proportion_remoteless = 0.50
reduction_efficiency_remote_work = 0.2
cost_healthcare = 4700 # Cost of healthcare per day
severeforms = 0.1 #Pourcentage of severe forms in the positive tests


optim = optimisationcovid(costs,normal_gdp,N,lengthsimulation,n_scenarios, initial_infections, average_remaining_working_years, proportion_work_infection,cost_healthcare, severeforms,proportion_remoteless,reduction_efficiency_remote_work, params)
startsolution = optim.startsolution()
print("start solution",startsolution)

thePareto = vns.vns(
    optim,
    [startsolution],   
    maxNeighborhood=100, 
    numberOfNeighbors=20, 
    archiveInputFile='optimPareto_5days.pickle',
    pickleOutputFile='optimPareto_5days.pickle',
)

print(f'Number of pareto solutions: {len(thePareto.pareto)}')
print(f'Pareto solutions: {thePareto.pareto}')
 

for p in thePareto.pareto.keys():
    obj = [f'{t}: {r} ' for t, r in zip(p.objectivesNames, p.objectives)]
    print(f'{p} {obj}')

start solution [ 0 20  1  1  1  1  1  1  1  1  1]
Processing period  1900-01-01 00:00:00
Processing period  1900-01-01 01:00:00
Processing period  1900-01-01 02:00:00
Processing period  1900-01-01 03:00:00
Processing period  1900-01-01 04:00:00
Processing period  1900-01-01 05:00:00
Processing period  1900-01-01 06:00:00
Processing period  1900-01-01 07:00:00
Processing period  1900-01-01 08:00:00
Processing period  1900-01-01 09:00:00
Processing period  1900-01-01 10:00:00
Processing period  1900-01-01 11:00:00
Processing period  1900-01-01 12:00:00
Processing period  1900-01-01 13:00:00
Processing period  1900-01-01 14:00:00
Processing period  1900-01-01 15:00:00
Processing period  1900-01-01 16:00:00
Processing period  1900-01-01 17:00:00
Processing period  1900-01-01 18:00:00
Processing period  1900-01-01 19:00:00
Processing period  1900-01-01 20:00:00
Processing period  1900-01-01 21:00:00
Processing period  1900-01-01 22:00:00
Processing period  1900-01-01 23:00:00
Loading activi

/Users/antoinedebouille/Documents/EPFL/MASTER/MA2/Semester project/ABM model/Covid_restriction_Github_version/abm/model.py:250: RuntimeWarning: invalid value encountered in divide
  infected_fractions = infected_visitors / visitors


Day  1
Day  2
Day  3
Day  4
Day  5
Day  6
Day  7
Day  8
Day  9
Day  10
Day  11
Day  12
Day  13
Day  14
Day  15
Day  16
Day  17
Day  18
Day  19
Day  20
Simulation ended. 
EVALUATE
[693138.6, 396529.50558]
20
Scenario n°  1
Scenario n°  1
Scenario n°  1
Scenario n°  1
Scenario n°  1
Scenario n°  1
Scenario n°  1
Scenario n°  1
Scenario n°  1
we are generating a neighbour 1
[16:15:30] < Warning >   Neighbor of size 1: generation failed
we are generating a neighbour 2
[16:15:30] < Warning >   Neighbor of size 2: generation failed
INCREASE SCHEDULE 0-9 YEARS OLD [0, 20, 4, 1, 1, 1, 1, 1, 1, 1, 1]
we are generating a neighbour 3
Processing period  1900-01-01 00:00:00
Processing period  1900-01-01 01:00:00
Processing period  1900-01-01 02:00:00
Processing period  1900-01-01 03:00:00
Processing period  1900-01-01 04:00:00
Processing period  1900-01-01 05:00:00
Processing period  1900-01-01 06:00:00
Processing period  1900-01-01 07:00:00
Processing period  1900-01-01 08:00:00
Processing period 

/Users/antoinedebouille/Documents/EPFL/MASTER/MA2/Semester project/ABM model/Covid_restriction_Github_version/abm/model.py:250: RuntimeWarning: invalid value encountered in divide
  infected_fractions = infected_visitors / visitors


Day  1
Day  2
Day  3
Day  4
Day  5
Day  6
Day  7
Day  8
Day  9
Day  10
Day  11
Day  12
Day  13
Day  14
Day  15
Day  16
Day  17
Day  18
Day  19
Day  20
Simulation ended. 
EVALUATE
[683824.9, 391346.76058]
20
Scenario n°  4
Scenario n°  1
Scenario n°  1
Scenario n°  1
Scenario n°  1
Scenario n°  1
Scenario n°  1
Scenario n°  1
Scenario n°  1
we are generating a neighbour 1
[16:19:16] < Warning >   Neighbor of size 1: generation failed
INCREASE SCHEDULE 50-59 YEARS OLD [0, 20, 1, 1, 1, 1, 1, 2, 1, 1, 1]
we are generating a neighbour 1
Processing period  1900-01-01 00:00:00
Processing period  1900-01-01 01:00:00
Processing period  1900-01-01 02:00:00
Processing period  1900-01-01 03:00:00
Processing period  1900-01-01 04:00:00
Processing period  1900-01-01 05:00:00
Processing period  1900-01-01 06:00:00
Processing period  1900-01-01 07:00:00
Processing period  1900-01-01 08:00:00
Processing period  1900-01-01 09:00:00
Processing period  1900-01-01 10:00:00
Processing period  1900-01-01 11:

...

/Users/antoinedebouille/Documents/EPFL/MASTER/MA2/Semester project/ABM model/Covid_restriction_Github_version/abm/model.py:250: RuntimeWarning: invalid value encountered in divide
  infected_fractions = infected_visitors / visitors


Day  1
Day  2
Day  3
Day  4
Day  5
Day  6
Day  7
Day  8
Day  9
Day  10
Day  11
Day  12
Day  13
Day  14
Day  15
Day  16
Day  17
Day  18
Day  19
Day  20
Simulation ended. 
EVALUATE
[43952.7, 51956.53113]
20
Scenario n°  1
Scenario n°  4
Scenario n°  1
Scenario n°  1
Scenario n°  4
Scenario n°  1
Scenario n°  1
Scenario n°  1
Scenario n°  1
we are generating a neighbour 1
[17:34:05] < Warning >   Neighbor of size 1: generation failed
we are generating a neighbour 4
[17:34:05] < Warning >   Neighbor of size 4: generation failed
we are generating a neighbour 5
[17:34:05] < Warning >   Neighbor of size 5: generation failed
INCREASE SCHEDULE 0-9 YEARS OLD [0, 20, 3, 4, 1, 1, 4, 1, 1, 1, 1]
we are generating a neighbour 2
Processing period  1900-01-01 00:00:00
Processing period  1900-01-01 01:00:00
Processing period  1900-01-01 02:00:00
Processing period  1900-01-01 03:00:00
Processing period  1900-01-01 04:00:00
Processing period  1900-01-01 05:00:00
Processing period  1900-01-01 06:00:00
Pro

/Users/antoinedebouille/Documents/EPFL/MASTER/MA2/Semester project/ABM model/Covid_restriction_Github_version/abm/model.py:250: RuntimeWarning: invalid value encountered in divide
  infected_fractions = infected_visitors / visitors


Day  1
Day  2
Day  3
Day  4
Day  5
Day  6
Day  7
Day  8
Day  9
Day  10
Day  11
Day  12
Day  13
Day  14
Day  15
Day  16
Day  17
Day  18
Day  19
Day  20
Simulation ended. 
EVALUATE
[53526.0, 60864.82349]
INCREASE SCHEDULE 80+ YEARS OLD [0, 20, 1, 4, 1, 1, 4, 1, 1, 1, 3]
we are generating a neighbour 2


FileNotFoundError: [Errno 2] No such file or directory: 'data/abm/vaud/prepared/scenarios/scenario_3/vaud_period_activities_80+.csv.gz'